# HNSW Algorithm Examples in FAISS

This notebook demonstrates the HNSW (Hierarchical Navigable Small World) algorithm in FAISS with various parameter combinations.

## Key Parameters

- **M**: Number of neighbors per node (default: 32). Higher M = better recall, more memory, slower search.
- **efConstruction**: Search depth during index construction (default: 40). Higher = better graph quality, slower build.
- **efSearch**: Search depth during query (default: 16). Higher = better recall, slower queries.

We'll explore how these parameters affect:
1. Search recall (accuracy)
2. Search time
3. Build time
4. Memory usage

In [ ]:
import numpy as np
import faiss
import time
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

## 1. Dataset Generation

We'll create a synthetic dataset for our experiments.

In [ ]:
def generate_dataset(nb, nq, d):
    """
    Generate random database and query vectors.
    
    Args:
        nb: Number of database vectors
        nq: Number of query vectors  
        d: Vector dimension
    
    Returns:
        xb: Database vectors (nb x d)
        xq: Query vectors (nq x d)
    """
    xb = np.random.random((nb, d)).astype('float32')
    xq = np.random.random((nq, d)).astype('float32')
    return xb, xq

# Dataset parameters
nb = 100000   # Database size
nq = 1000     # Number of queries
d = 128       # Vector dimension
k = 10        # Number of nearest neighbors to find

print(f"Generating dataset: {nb:,} database vectors, {nq:,} queries, dimension {d}")
xb, xq = generate_dataset(nb, nq, d)
print(f"Database shape: {xb.shape}, Query shape: {xq.shape}")

## 2. Ground Truth Computation

Compute exact nearest neighbors using brute-force search for recall evaluation.

In [ ]:
def compute_ground_truth(xb, xq, k):
    """Compute exact nearest neighbors using IndexFlatL2."""
    index_flat = faiss.IndexFlatL2(xb.shape[1])
    index_flat.add(xb)
    distances_gt, labels_gt = index_flat.search(xq, k)
    return distances_gt, labels_gt

def compute_recall(labels_gt, labels_approx, k):
    """Compute recall@k: fraction of true neighbors found."""
    n = labels_gt.shape[0]
    recall = 0.0
    for i in range(n):
        gt_set = set(labels_gt[i, :k])
        approx_set = set(labels_approx[i, :k])
        recall += len(gt_set & approx_set) / k
    return recall / n

print("Computing ground truth with brute-force search...")
start = time.time()
distances_gt, labels_gt = compute_ground_truth(xb, xq, k)
gt_time = time.time() - start
print(f"Ground truth computed in {gt_time:.2f} seconds")

## 3. Helper Functions for HNSW Experiments

In [ ]:
def build_hnsw_index(xb, M, efConstruction):
    """
    Build HNSW index with given parameters.
    
    Returns:
        index: Built HNSW index
        build_time: Time taken to build (seconds)
    """
    d = xb.shape[1]
    index = faiss.IndexHNSWFlat(d, M)
    index.hnsw.efConstruction = efConstruction
    
    start = time.time()
    index.add(xb)
    build_time = time.time() - start
    
    return index, build_time

def search_hnsw_index(index, xq, k, efSearch):
    """
    Search HNSW index with given efSearch.
    
    Returns:
        distances: Distance array
        labels: Label array
        search_time: Time taken to search (seconds)
    """
    index.hnsw.efSearch = efSearch
    
    start = time.time()
    distances, labels = index.search(xq, k)
    search_time = time.time() - start
    
    return distances, labels, search_time

def estimate_memory_usage(index):
    """Estimate memory usage of HNSW index in MB."""
    # Vector storage: ntotal * d * 4 bytes
    vector_mem = index.ntotal * index.d * 4
    
    # Graph structure estimation
    # Level 0: 2*M neighbors, Higher levels: M neighbors
    M = index.hnsw.nb_neighbors(0) // 2
    graph_mem = index.ntotal * 2 * M * 4  # 4 bytes per neighbor ID
    
    total_mb = (vector_mem + graph_mem) / (1024 * 1024)
    return total_mb

## 4. Experiment 1: Effect of M (Number of Neighbors)

M controls the number of bidirectional links per element. Higher M means:
- Better recall but slower search
- More memory usage
- Slower index construction

In [ ]:
# Test different values of M
M_values = [4, 8, 16, 32, 48, 64]
efConstruction_fixed = 40
efSearch_fixed = 32

results_M = []

print("Experiment 1: Varying M")
print(f"Fixed efConstruction={efConstruction_fixed}, efSearch={efSearch_fixed}\n")
print(f"{'M':>6} {'Build(s)':>10} {'Search(s)':>10} {'Recall':>10} {'Memory(MB)':>12}")
print("-" * 52)

for M in M_values:
    # Build index
    index, build_time = build_hnsw_index(xb, M, efConstruction_fixed)
    
    # Search
    distances, labels, search_time = search_hnsw_index(index, xq, k, efSearch_fixed)
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    memory = estimate_memory_usage(index)
    
    results_M.append({
        'M': M,
        'build_time': build_time,
        'search_time': search_time,
        'recall': recall,
        'memory_mb': memory
    })
    
    print(f"{M:>6} {build_time:>10.2f} {search_time:>10.4f} {recall:>10.4f} {memory:>12.1f}")

df_M = pd.DataFrame(results_M)

In [ ]:
# Visualize effect of M
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recall vs M
ax1 = axes[0, 0]
ax1.plot(df_M['M'], df_M['recall'], 'o-', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('M (neighbors per node)')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs M')
ax1.set_ylim([0.5, 1.02])
ax1.grid(True, alpha=0.3)

# Search time vs M
ax2 = axes[0, 1]
ax2.plot(df_M['M'], df_M['search_time'] * 1000, 'o-', linewidth=2, markersize=8, color='#e74c3c')
ax2.set_xlabel('M (neighbors per node)')
ax2.set_ylabel('Search Time (ms)')
ax2.set_title('Search Time vs M')
ax2.grid(True, alpha=0.3)

# Build time vs M
ax3 = axes[1, 0]
ax3.plot(df_M['M'], df_M['build_time'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax3.set_xlabel('M (neighbors per node)')
ax3.set_ylabel('Build Time (s)')
ax3.set_title('Build Time vs M')
ax3.grid(True, alpha=0.3)

# Memory vs M
ax4 = axes[1, 1]
ax4.plot(df_M['M'], df_M['memory_mb'], 'o-', linewidth=2, markersize=8, color='#9b59b6')
ax4.set_xlabel('M (neighbors per node)')
ax4.set_ylabel('Memory (MB)')
ax4.set_title('Memory Usage vs M')
ax4.grid(True, alpha=0.3)

plt.suptitle(f'Effect of M (efConstruction={efConstruction_fixed}, efSearch={efSearch_fixed})', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Experiment 2: Effect of efConstruction

efConstruction controls the search depth during index construction. Higher values:
- Build a higher quality graph
- Take longer to build
- Generally improve recall (especially for lower M)

In [ ]:
# Test different values of efConstruction
efConstruction_values = [10, 20, 40, 80, 100, 200]
M_fixed = 32
efSearch_fixed = 32

results_efC = []

print("Experiment 2: Varying efConstruction")
print(f"Fixed M={M_fixed}, efSearch={efSearch_fixed}\n")
print(f"{'efConstruction':>14} {'Build(s)':>10} {'Search(s)':>10} {'Recall':>10}")
print("-" * 48)

for efC in efConstruction_values:
    # Build index
    index, build_time = build_hnsw_index(xb, M_fixed, efC)
    
    # Search
    distances, labels, search_time = search_hnsw_index(index, xq, k, efSearch_fixed)
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    
    results_efC.append({
        'efConstruction': efC,
        'build_time': build_time,
        'search_time': search_time,
        'recall': recall
    })
    
    print(f"{efC:>14} {build_time:>10.2f} {search_time:>10.4f} {recall:>10.4f}")

df_efC = pd.DataFrame(results_efC)

In [ ]:
# Visualize effect of efConstruction
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Recall vs efConstruction
ax1 = axes[0]
ax1.plot(df_efC['efConstruction'], df_efC['recall'], 'o-', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('efConstruction')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs efConstruction')
ax1.set_ylim([0.8, 1.02])
ax1.grid(True, alpha=0.3)

# Build time vs efConstruction
ax2 = axes[1]
ax2.plot(df_efC['efConstruction'], df_efC['build_time'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax2.set_xlabel('efConstruction')
ax2.set_ylabel('Build Time (s)')
ax2.set_title('Build Time vs efConstruction')
ax2.grid(True, alpha=0.3)

# Search time vs efConstruction (should be relatively constant)
ax3 = axes[2]
ax3.plot(df_efC['efConstruction'], df_efC['search_time'] * 1000, 'o-', linewidth=2, markersize=8, color='#e74c3c')
ax3.set_xlabel('efConstruction')
ax3.set_ylabel('Search Time (ms)')
ax3.set_title('Search Time vs efConstruction\n(should be stable)')
ax3.grid(True, alpha=0.3)

plt.suptitle(f'Effect of efConstruction (M={M_fixed}, efSearch={efSearch_fixed})', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Experiment 3: Effect of efSearch

efSearch controls the search depth at query time. This is the main trade-off:
- Higher efSearch = better recall but slower search
- Can be adjusted without rebuilding the index

In [ ]:
# Test different values of efSearch
efSearch_values = [8, 16, 32, 64, 128, 256, 512]
M_fixed = 32
efConstruction_fixed = 40

# Build index once
print(f"Building index with M={M_fixed}, efConstruction={efConstruction_fixed}...")
index, build_time = build_hnsw_index(xb, M_fixed, efConstruction_fixed)
print(f"Index built in {build_time:.2f}s\n")

results_efS = []

print("Experiment 3: Varying efSearch")
print(f"{'efSearch':>10} {'Search(ms)':>12} {'Recall':>10} {'QPS':>12}")
print("-" * 48)

for efS in efSearch_values:
    # Search
    distances, labels, search_time = search_hnsw_index(index, xq, k, efS)
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    qps = nq / search_time  # Queries per second
    
    results_efS.append({
        'efSearch': efS,
        'search_time_ms': search_time * 1000,
        'recall': recall,
        'qps': qps
    })
    
    print(f"{efS:>10} {search_time*1000:>12.2f} {recall:>10.4f} {qps:>12.0f}")

df_efS = pd.DataFrame(results_efS)

In [ ]:
# Visualize effect of efSearch
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Recall vs efSearch
ax1 = axes[0]
ax1.plot(df_efS['efSearch'], df_efS['recall'], 'o-', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('efSearch')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs efSearch')
ax1.set_xscale('log', base=2)
ax1.set_ylim([0.6, 1.02])
ax1.grid(True, alpha=0.3)

# Search time vs efSearch
ax2 = axes[1]
ax2.plot(df_efS['efSearch'], df_efS['search_time_ms'], 'o-', linewidth=2, markersize=8, color='#e74c3c')
ax2.set_xlabel('efSearch')
ax2.set_ylabel('Search Time (ms)')
ax2.set_title('Search Time vs efSearch')
ax2.set_xscale('log', base=2)
ax2.grid(True, alpha=0.3)

# QPS vs efSearch
ax3 = axes[2]
ax3.plot(df_efS['efSearch'], df_efS['qps'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax3.set_xlabel('efSearch')
ax3.set_ylabel('Queries Per Second')
ax3.set_title('Throughput vs efSearch')
ax3.set_xscale('log', base=2)
ax3.grid(True, alpha=0.3)

plt.suptitle(f'Effect of efSearch (M={M_fixed}, efConstruction={efConstruction_fixed})', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Recall-QPS trade-off curve (Pareto frontier)
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(df_efS['recall'], df_efS['qps'], 'o-', linewidth=2, markersize=10, color='#8e44ad')

# Annotate points with efSearch values
for i, row in df_efS.iterrows():
    ax.annotate(f"ef={int(row['efSearch'])}", 
                (row['recall'], row['qps']),
                textcoords="offset points", 
                xytext=(5, 5), 
                fontsize=9)

ax.set_xlabel('Recall@10', fontsize=12)
ax.set_ylabel('Queries Per Second (QPS)', fontsize=12)
ax.set_title('Recall vs Throughput Trade-off\n(Pareto Frontier for efSearch)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Experiment 4: Combined Parameter Grid Search

Let's explore the combined effect of M and efSearch at query time.

In [ ]:
# Grid search over M and efSearch
M_grid = [8, 16, 32, 64]
efSearch_grid = [16, 32, 64, 128, 256]
efConstruction_fixed = 64

# Pre-build indexes for each M
indexes = {}
print("Building indexes...")
for M in M_grid:
    index, build_time = build_hnsw_index(xb, M, efConstruction_fixed)
    indexes[M] = index
    print(f"  M={M}: built in {build_time:.2f}s")

# Run grid search
grid_results = []

print("\nRunning grid search...")
for M in M_grid:
    index = indexes[M]
    for efS in efSearch_grid:
        distances, labels, search_time = search_hnsw_index(index, xq, k, efS)
        recall = compute_recall(labels_gt, labels, k)
        qps = nq / search_time
        
        grid_results.append({
            'M': M,
            'efSearch': efS,
            'recall': recall,
            'search_time_ms': search_time * 1000,
            'qps': qps
        })

df_grid = pd.DataFrame(grid_results)
print("\nGrid search complete!")

In [ ]:
# Visualize grid results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

colors = plt.cm.viridis(np.linspace(0, 1, len(M_grid)))

# Recall curves for different M
ax1 = axes[0]
for i, M in enumerate(M_grid):
    data = df_grid[df_grid['M'] == M]
    ax1.plot(data['efSearch'], data['recall'], 'o-', 
             linewidth=2, markersize=8, color=colors[i], label=f'M={M}')
ax1.set_xlabel('efSearch')
ax1.set_ylabel('Recall@10')
ax1.set_title('Recall vs efSearch for Different M')
ax1.set_xscale('log', base=2)
ax1.set_ylim([0.5, 1.02])
ax1.legend()
ax1.grid(True, alpha=0.3)

# Recall vs QPS for different M
ax2 = axes[1]
for i, M in enumerate(M_grid):
    data = df_grid[df_grid['M'] == M]
    ax2.plot(data['recall'], data['qps'], 'o-', 
             linewidth=2, markersize=8, color=colors[i], label=f'M={M}')
ax2.set_xlabel('Recall@10')
ax2.set_ylabel('Queries Per Second')
ax2.set_title('Recall-Throughput Trade-off')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle(f'Combined Effect of M and efSearch (efConstruction={efConstruction_fixed})', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap of recall and QPS values
pivot_recall = df_grid.pivot(index='M', columns='efSearch', values='recall')
pivot_qps = df_grid.pivot(index='M', columns='efSearch', values='qps')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Recall heatmap
im1 = axes[0].imshow(pivot_recall.values, cmap='RdYlGn', aspect='auto', vmin=0.5, vmax=1.0)
axes[0].set_xticks(range(len(efSearch_grid)))
axes[0].set_xticklabels(efSearch_grid)
axes[0].set_yticks(range(len(M_grid)))
axes[0].set_yticklabels(M_grid)
axes[0].set_xlabel('efSearch')
axes[0].set_ylabel('M')
axes[0].set_title('Recall@10 Heatmap')

# Add text annotations
for i in range(len(M_grid)):
    for j in range(len(efSearch_grid)):
        text = axes[0].text(j, i, f'{pivot_recall.values[i, j]:.3f}',
                           ha='center', va='center', color='black', fontsize=10)

fig.colorbar(im1, ax=axes[0])

# QPS heatmap
im2 = axes[1].imshow(pivot_qps.values, cmap='YlOrRd_r', aspect='auto')
axes[1].set_xticks(range(len(efSearch_grid)))
axes[1].set_xticklabels(efSearch_grid)
axes[1].set_yticks(range(len(M_grid)))
axes[1].set_yticklabels(M_grid)
axes[1].set_xlabel('efSearch')
axes[1].set_ylabel('M')
axes[1].set_title('QPS Heatmap')

# Add text annotations
for i in range(len(M_grid)):
    for j in range(len(efSearch_grid)):
        text = axes[1].text(j, i, f'{pivot_qps.values[i, j]:.0f}',
                           ha='center', va='center', color='black', fontsize=10)

fig.colorbar(im2, ax=axes[1])

plt.suptitle('Parameter Grid Search Results', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Experiment 5: Extreme Parameter Configurations

Let's explore some edge cases with very low and very high parameter values.

In [ ]:
# Extreme configurations
extreme_configs = [
    {'name': 'Minimal', 'M': 4, 'efConstruction': 10, 'efSearch': 8},
    {'name': 'Low-Memory', 'M': 8, 'efConstruction': 20, 'efSearch': 16},
    {'name': 'Balanced', 'M': 32, 'efConstruction': 40, 'efSearch': 32},
    {'name': 'High-Recall', 'M': 48, 'efConstruction': 100, 'efSearch': 128},
    {'name': 'Maximum', 'M': 64, 'efConstruction': 200, 'efSearch': 256},
]

extreme_results = []

print("Experiment 5: Extreme Parameter Configurations\n")
print(f"{'Config':>12} {'M':>4} {'efC':>6} {'efS':>6} {'Build(s)':>10} {'Search(ms)':>12} {'Recall':>8} {'Mem(MB)':>10}")
print("-" * 82)

for config in extreme_configs:
    # Build index
    index, build_time = build_hnsw_index(xb, config['M'], config['efConstruction'])
    
    # Search
    distances, labels, search_time = search_hnsw_index(index, xq, k, config['efSearch'])
    
    # Compute metrics
    recall = compute_recall(labels_gt, labels, k)
    memory = estimate_memory_usage(index)
    
    extreme_results.append({
        'name': config['name'],
        'M': config['M'],
        'efConstruction': config['efConstruction'],
        'efSearch': config['efSearch'],
        'build_time': build_time,
        'search_time_ms': search_time * 1000,
        'recall': recall,
        'memory_mb': memory
    })
    
    print(f"{config['name']:>12} {config['M']:>4} {config['efConstruction']:>6} {config['efSearch']:>6} "
          f"{build_time:>10.2f} {search_time*1000:>12.2f} {recall:>8.4f} {memory:>10.1f}")

df_extreme = pd.DataFrame(extreme_results)

In [ ]:
# Visualize extreme configurations
fig, ax = plt.subplots(figsize=(12, 7))

# Create a scatter plot with size based on memory and color based on search time
scatter = ax.scatter(df_extreme['recall'], df_extreme['build_time'],
                     s=df_extreme['memory_mb'] * 2,  # Size based on memory
                     c=df_extreme['search_time_ms'],  # Color based on search time
                     cmap='coolwarm', alpha=0.7, edgecolors='black', linewidth=2)

# Add labels for each point
for i, row in df_extreme.iterrows():
    ax.annotate(f"{row['name']}\nM={row['M']}, efC={row['efConstruction']}, efS={row['efSearch']}",
                (row['recall'], row['build_time']),
                textcoords="offset points",
                xytext=(10, 5),
                fontsize=9,
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

ax.set_xlabel('Recall@10', fontsize=12)
ax.set_ylabel('Build Time (s)', fontsize=12)
ax.set_title('Extreme Configurations Comparison\n(Size = Memory, Color = Search Time)', 
             fontsize=14, fontweight='bold')

cbar = plt.colorbar(scatter)
cbar.set_label('Search Time (ms)', fontsize=11)

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Experiment 6: Comparing with Brute Force

Let's compare HNSW performance against brute-force search.

In [ ]:
# Brute force baseline
print("Comparing HNSW with Brute Force (IndexFlatL2)\n")

# Build brute force index
index_flat = faiss.IndexFlatL2(d)
start = time.time()
index_flat.add(xb)
flat_build_time = time.time() - start

# Search with brute force
start = time.time()
D_flat, I_flat = index_flat.search(xq, k)
flat_search_time = time.time() - start

flat_qps = nq / flat_search_time

print(f"IndexFlatL2 (Brute Force):")
print(f"  Build time: {flat_build_time:.3f}s")
print(f"  Search time: {flat_search_time*1000:.2f}ms")
print(f"  QPS: {flat_qps:.0f}")
print(f"  Recall: 1.0000 (exact)")

# HNSW with good configuration
hnsw_config = {'M': 32, 'efConstruction': 64, 'efSearch': 64}
index_hnsw, hnsw_build_time = build_hnsw_index(xb, hnsw_config['M'], hnsw_config['efConstruction'])
D_hnsw, I_hnsw, hnsw_search_time = search_hnsw_index(index_hnsw, xq, k, hnsw_config['efSearch'])
hnsw_recall = compute_recall(I_flat, I_hnsw, k)
hnsw_qps = nq / hnsw_search_time

print(f"\nIndexHNSWFlat (M={hnsw_config['M']}, efC={hnsw_config['efConstruction']}, efS={hnsw_config['efSearch']}):")
print(f"  Build time: {hnsw_build_time:.3f}s")
print(f"  Search time: {hnsw_search_time*1000:.2f}ms")
print(f"  QPS: {hnsw_qps:.0f}")
print(f"  Recall: {hnsw_recall:.4f}")

print(f"\nSpeedup: {hnsw_qps/flat_qps:.1f}x faster at {hnsw_recall*100:.1f}% recall")

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

methods = ['Brute Force', f'HNSW\n(M={hnsw_config["M"]})']
colors = ['#3498db', '#2ecc71']

# Build time
build_times = [flat_build_time, hnsw_build_time]
axes[0].bar(methods, build_times, color=colors, edgecolor='black', linewidth=1.5)
axes[0].set_ylabel('Build Time (s)')
axes[0].set_title('Build Time Comparison')
for i, v in enumerate(build_times):
    axes[0].text(i, v + 0.02, f'{v:.2f}s', ha='center', fontweight='bold')

# QPS
qps_values = [flat_qps, hnsw_qps]
axes[1].bar(methods, qps_values, color=colors, edgecolor='black', linewidth=1.5)
axes[1].set_ylabel('Queries Per Second')
axes[1].set_title('Search Throughput Comparison')
for i, v in enumerate(qps_values):
    axes[1].text(i, v + 100, f'{v:.0f}', ha='center', fontweight='bold')

# Recall
recall_values = [1.0, hnsw_recall]
axes[2].bar(methods, recall_values, color=colors, edgecolor='black', linewidth=1.5)
axes[2].set_ylabel('Recall@10')
axes[2].set_title('Recall Comparison')
axes[2].set_ylim([0, 1.1])
for i, v in enumerate(recall_values):
    axes[2].text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

plt.suptitle('HNSW vs Brute Force Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 10. Summary and Recommendations

Based on our experiments, here are the key findings:

In [ ]:
print("="*70)
print("HNSW Parameter Tuning Guidelines")
print("="*70)

print("""
1. M (Number of neighbors per node):
   - Higher M → Better recall, more memory, slower search/build
   - Recommended range: 16-64
   - Default: 32 is a good starting point
   - Use lower M (8-16) for memory-constrained scenarios
   - Use higher M (48-64) when recall is critical

2. efConstruction (Build-time search depth):
   - Higher efConstruction → Better graph quality, slower build
   - Recommended range: 40-200
   - Rule of thumb: efConstruction should be >= M
   - Set it once and forget (doesn't affect search speed)
   - Use higher values (100+) for smaller M values

3. efSearch (Query-time search depth):
   - This is your main recall/speed trade-off knob
   - Higher efSearch → Better recall, slower search
   - Recommended range: k to 500
   - Must be >= k (number of results requested)
   - Can be adjusted at runtime without rebuilding index

Common Configurations:
-------------------------------------------
| Use Case          | M  | efC | efSearch |
|--------------------|----|----|----------|
| Low memory        | 8  | 40 | 16-32    |
| Balanced          | 32 | 64 | 32-64    |
| High recall       | 48 | 100| 128-256  |
| Maximum recall    | 64 | 200| 256-512  |
-------------------------------------------
""")

# Show best configurations from our experiments
print("\nBest configurations from our experiments:")
print("-" * 50)

# Find configurations with >95% recall
high_recall = df_grid[df_grid['recall'] > 0.95].sort_values('qps', ascending=False)
if len(high_recall) > 0:
    best = high_recall.iloc[0]
    print(f"Best for >95% recall: M={int(best['M'])}, efSearch={int(best['efSearch'])}")
    print(f"  Recall: {best['recall']:.4f}, QPS: {best['qps']:.0f}")

# Find fastest configuration with >90% recall
fast_good = df_grid[df_grid['recall'] > 0.90].sort_values('qps', ascending=False)
if len(fast_good) > 0:
    best = fast_good.iloc[0]
    print(f"\nFastest with >90% recall: M={int(best['M'])}, efSearch={int(best['efSearch'])}")
    print(f"  Recall: {best['recall']:.4f}, QPS: {best['qps']:.0f}")

## 11. Interactive Parameter Explorer

Use this cell to experiment with your own parameter combinations.

In [ ]:
def test_hnsw_config(M, efConstruction, efSearch, xb, xq, labels_gt, k):
    """
    Test a specific HNSW configuration and print results.
    """
    print(f"Testing HNSW with M={M}, efConstruction={efConstruction}, efSearch={efSearch}")
    print("-" * 60)
    
    # Build
    index, build_time = build_hnsw_index(xb, M, efConstruction)
    print(f"Build time: {build_time:.2f}s")
    
    # Search
    distances, labels, search_time = search_hnsw_index(index, xq, k, efSearch)
    print(f"Search time: {search_time*1000:.2f}ms for {len(xq)} queries")
    
    # Metrics
    recall = compute_recall(labels_gt, labels, k)
    qps = len(xq) / search_time
    memory = estimate_memory_usage(index)
    
    print(f"Recall@{k}: {recall:.4f}")
    print(f"QPS: {qps:.0f}")
    print(f"Estimated memory: {memory:.1f} MB")
    
    return index, recall, qps

# Example: Try your own configuration!
# Modify these parameters and run the cell
my_M = 24
my_efConstruction = 80
my_efSearch = 48

test_hnsw_config(my_M, my_efConstruction, my_efSearch, xb, xq, labels_gt, k)